## Building a NB Classifier from scrach


**IMPORTANT**: Make sure this first cell let's chdir point to the project's folder with name  "NaiveBayesClassifier"
#Make imports work 




In [5]:
import os

print(os.getcwd()) #show current dir (should end with NaiveBayesClassier)

#if not:
if os.getcwd().split('\\')[-1] != 'NaiveBayesClassifier' :
    os.chdir('c:\\Users\\Benjamin\\OneDrive\\Desktop\\CPSC 481-07 (AI)\\Code Examples\\NaiveBayesClassifier\\') #maybe adjust this path to point at right dir

c:\Users\Benjamin\OneDrive\Desktop\CPSC 481-07 (AI)\Code Examples\NaiveBayesClassifier



this CountVectorizer is quite useful. It counts the words and represents

- each sentence / email as a row
- each word / term as a column

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

texts = ["blue house and blue window", "window window green house", "green house and blue blue"]

vectorizer = CountVectorizer()
term_count_matrix = vectorizer.fit_transform(texts)

# The unique words / column names
print(vectorizer.get_feature_names()) #in our sklearn version it is get_feature_names, in newer versions it's get_feature_names_out

# Each word"s amount of occurences in texts
print(term_count_matrix.toarray())



['and', 'blue', 'green', 'house', 'window']
[[1 2 0 1 1]
 [0 0 1 1 2]
 [1 2 1 1 0]]


**Looking at our data in data_cooked.csv which looks something like this (example)**


urgent contact last weekend draw show prize call claim code valid,1

get dump heap mom decid come low bore,0

ok lor soni ericsson salesman ask shuhui say quit gd use consid,0

privat account statement show un redeem point call identifi code expir,1


In [7]:
#First get the data
import numpy as np


#I took this from our prepared data and adjusted a bit to illustrate better
data= np.array([
("free claim call code", 1),
("heap mom bore",0),
("ok salesman quit",0),
("urgent call free identifi code",1)
])

mails = data[:,0] #all rows, 0th column
labels = data[:,1] #all rows, 1st column


# Second: use the vectorizer on the mails
vectorizer = CountVectorizer()
word_count_matrix = vectorizer.fit_transform(mails)
print(vectorizer.get_feature_names()) # Column Names 
print('Word Count Matrix looks like this:\n', word_count_matrix.toarray())




['bore', 'call', 'claim', 'code', 'free', 'heap', 'identifi', 'mom', 'ok', 'quit', 'salesman', 'urgent']
Word Count Matrix looks like this:
 [[0 1 1 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0]
 [0 1 0 1 1 0 1 0 0 0 0 1]]


as a next step, we can calculating:

for each word (meaning for each column): Count how often the word appears in a spam sentence and then calculate its probability as 

Occurences in Spam Mails  /  Amount all mails

In [8]:
probabilities = {}


number_all_mails = len(word_count_matrix.toarray())
for index,word in enumerate(vectorizer.get_feature_names()):
    column = word_count_matrix[:,index]
    count = np.count_nonzero(column[column>0]) # column[column>0] makes column a 1D array of True and False values 
    probabilities[word] = count / number_all_mails

probabilities


{'bore': 0.25,
 'call': 0.5,
 'claim': 0.25,
 'code': 0.5,
 'free': 0.5,
 'heap': 0.25,
 'identifi': 0.25,
 'mom': 0.25,
 'ok': 0.25,
 'quit': 0.25,
 'salesman': 0.25,
 'urgent': 0.25}

See how words like call, code, free are rated higher than other words

it's still very simple the way it is but could be a start 

## Building the NBC from scratch

In the following code cells I tried to code the Naive Bayes Calssifier from our lecture, starting at slide number 28 in "Reasoning with uncertainty.pptx".

This preparing the data and 2 attempts of implementing the classifier.

**Each of them is commented out** since later on in the project, we wrapped the code into method definitions. This way, not every notebook run is slowed by running the next 3 code blocks.



In [9]:
#Start by getting some data and making it usable:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

"""
data_path = "Data/Preprocessing/data_cooked.csv"

dataframe = pd.read_csv(data_path)

# somehow there are missing values somewhere, may have to check Preprocessing.ipynb again
# very important to reset index! Because pandas would otherwise memorize the old indices which is bad for what is about to come
dataframe = dataframe.dropna(subset=['Message']).reset_index(drop=True)

mails = dataframe['Message']
labels = dataframe['Category']

#cv = CountVectorizer()
#word_count_matrix = cv.fit_transform(mails.to_numpy()).toarray()
"""

'\ndata_path = "Data/Preprocessing/data_cooked.csv"\n\ndataframe = pd.read_csv(data_path)\n\n# somehow there are missing values somewhere, may have to check Preprocessing.ipynb again\n# very important to reset index! Because pandas would otherwise memorize the old indices which is bad for what is about to come\ndataframe = dataframe.dropna(subset=[\'Message\']).reset_index(drop=True)\n\nmails = dataframe[\'Message\']\nlabels = dataframe[\'Category\']\n\n#cv = CountVectorizer()\n#word_count_matrix = cv.fit_transform(mails.to_numpy()).toarray()\n'

In [10]:
#1.Attempt
#CAUTION: This was my initial attempt which follows exactly what we did in the lecture on slide 32
#but it takes a long time to execute. See a below cell that uses numpy matrix operations

#(Commented this code because it's wrapped in a method later on andI don"t want
#this cell to run in each notebook run and take up extra time)

""" 

#Calculating P(SPAM) and P(not SPAM)
spam_count_total = len(labels.loc[labels==1])
not_spam_count_total = len(labels.loc[labels==0])
total = spam_count_total + not_spam_count_total

P_spam = spam_count_total / total
P_not_spam = not_spam_count_total / total

#Calculating the Conditional Probability for each word:


# An array that counts the values for each word
unique_word_count = len(word_count_matrix[0]) #how many elements does one row have + eunique words in corpus

words = cv.get_feature_names()
word_proba_array = np.zeros((2,unique_word_count)) 
#2 rows:
# FIRST ROW for the word's cond. probab for spam=yes
# SECOND ROW for the word's probab for spam=no
#(see slide 32 in lecture)

#Y columns: as many colums as unique words there are in data


#Laplace thing to not get 0-probabs
alpha = 1

#Get the idices for all spam rows and for all non spam rows
spam_rows_indices = labels[labels == 1].index.tolist()
not_spam_rows_indices = labels[labels == 0].index.tolist()

# Calculate the two conditional probabilities for each word
for word_idx in range(len(word_count_matrix[0])):
    
    #calculating for spam=yes rows
    for row_index in spam_rows_indices:
        word_count = word_count_matrix[row_index][word_idx]
        if(word_count > 0): #only checks if the word appears, not how often
            word_proba_array[0][word_idx] += 1 / (spam_count_total)
            #-> word_proba_array[1] means writing to the row which is for concitional probabs for Spam=Yes
            #-> word_proba_array[1][word_idx] means we're writing to the index of the word we"re looking at right now
            #-> "+= 1 / spam_count_total" increases the probability by one each time a word is found for a spam=yes mail


    #calculating for spam=no rows
    for row_index in not_spam_rows_indices:
        word_count = word_count_matrix[row_index][word_idx]
        if(word_count > 0): #only checks if the word appears, not how often
            word_proba_array[1][word_idx] += 1 / (not_spam_count_total)
            #-> same as before but for spam=no mails



#COncatenating words with their probabilities

probabilities_df = pd.DataFrame({
    'Word': words,
    'P(Word|Spam)': word_proba_array[0],
    'P(Word|NotSpam)': word_proba_array[1]
})
"""


' \n\n#Calculating P(SPAM) and P(not SPAM)\nspam_count_total = len(labels.loc[labels==1])\nnot_spam_count_total = len(labels.loc[labels==0])\ntotal = spam_count_total + not_spam_count_total\n\nP_spam = spam_count_total / total\nP_not_spam = not_spam_count_total / total\n\n#Calculating the Conditional Probability for each word:\n\n\n# An array that counts the values for each word\nunique_word_count = len(word_count_matrix[0]) #how many elements does one row have + eunique words in corpus\n\nwords = cv.get_feature_names()\nword_proba_array = np.zeros((2,unique_word_count)) \n#2 rows:\n# FIRST ROW for the word\'s cond. probab for spam=yes\n# SECOND ROW for the word\'s probab for spam=no\n#(see slide 32 in lecture)\n\n#Y columns: as many colums as unique words there are in data\n\n\n#Laplace thing to not get 0-probabs\nalpha = 1\n\n#Get the idices for all spam rows and for all non spam rows\nspam_rows_indices = labels[labels == 1].index.tolist()\nnot_spam_rows_indices = labels[labels == 0]

(this next cell once crashed with a MemoryError for me. But after I rebooted my laptop, it had no more problems) 

In [11]:
# 2nd Attempt (Commented this code because it's wrapped in a method later on andI don"t want
#this cell to run in each notebook run and take up extra time)
"""
spam_count_total = np.sum(labels == 1)
not_spam_count_total = np.sum(labels == 0)
total = spam_count_total + not_spam_count_total

P_spam = spam_count_total / total
P_not_spam = not_spam_count_total / total

alpha = 1
unique_word_count = len(cv.get_feature_names())
words = np.array(cv.get_feature_names())

#instead of looping through the whole dataset counting word appearcances and adding probabilities, make 
#an array that counts each word's appearances over the whole dataset (also considers how often a word
# appears in a sentence and not only if or if it doean's appear as in the 1.attempt)
spam_word_counts = word_count_matrix[labels == 1].sum(axis=0)
not_spam_word_counts = word_count_matrix[labels == 0].sum(axis=0)

#Calculating the contiditional probalbilities by going  spam_word_counts/ total amount of spam word
# + it adds laplace smoothing as seen here:  https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece
word_proba_spam = (spam_word_counts + alpha) / (spam_word_counts.sum() + alpha * unique_word_count)
word_proba_not_spam = (not_spam_word_counts + alpha) / (not_spam_word_counts.sum() + alpha * unique_word_count)



probabilities_df = pd.DataFrame({
    'Word': words,
    'P(Word|Spam)': word_proba_spam,
    'P(Word|NotSpam)': word_proba_not_spam
})




"""


"\nspam_count_total = np.sum(labels == 1)\nnot_spam_count_total = np.sum(labels == 0)\ntotal = spam_count_total + not_spam_count_total\n\nP_spam = spam_count_total / total\nP_not_spam = not_spam_count_total / total\n\nalpha = 1\nunique_word_count = len(cv.get_feature_names())\nwords = np.array(cv.get_feature_names())\n\n#instead of looping through the whole dataset counting word appearcances and adding probabilities, make \n#an array that counts each word's appearances over the whole dataset (also considers how often a word\n# appears in a sentence and not only if or if it doean's appear as in the 1.attempt)\nspam_word_counts = word_count_matrix[labels == 1].sum(axis=0)\nnot_spam_word_counts = word_count_matrix[labels == 0].sum(axis=0)\n\n#Calculating the contiditional probalbilities by going  spam_word_counts/ total amount of spam word\n# + it adds laplace smoothing as seen here:  https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece\nword_proba_s

Now write this as a function:

In [12]:
def get_prior_probas(mails:pd.DataFrame, labels:pd.DataFrame):

    spam_count_total = np.sum(labels == 1)
    not_spam_count_total = np.sum(labels == 0)
    total = spam_count_total + not_spam_count_total

    P_spam = spam_count_total / total
    P_not_spam = not_spam_count_total / total

    ret_dict ={
        "P_spam" : P_spam, 
        "P_not_spam": P_not_spam
    } 
    return ret_dict


#### The following cell defines two methods: 
- **get_cond_probas_attempt_1()** that uses the code from the 1st attempt and 
- **get_cond_probas_attempt_2()** that uses the code from the 2nd attempt 


to calculate the conditional probabilities

In [13]:
def get_cond_probas_attempt_1(p_mails:pd.DataFrame, p_labels:pd.DataFrame):
    p_mails = p_mails.reset_index(drop=True)
    p_labels = p_labels.reset_index(drop=True)
    

    cv = CountVectorizer()
    word_count_matrix = cv.fit_transform(p_mails.to_numpy()).toarray()

    spam_count_total = np.sum(p_labels == 1)
    not_spam_count_total = np.sum(p_labels == 0)
    
    unique_word_count = len(word_count_matrix[0]) #how many elements does one row have + eunique words in corpus

    words = cv.get_feature_names()
    word_proba_array = np.zeros((2,unique_word_count))
    
    #Get the idices for all spam rows and for all non spam rows
    spam_rows_indices = p_labels[p_labels == 1].index.tolist()
    not_spam_rows_indices = p_labels[p_labels == 0].index.tolist()

    # Calculate the two conditional probabilities for each word
    for word_idx in range(len(word_count_matrix[0])):
        
        #calculating for spam=yes rows
        for row_index in spam_rows_indices:
            word_count = word_count_matrix[row_index][word_idx]
            if(word_count > 0): #only checks if the word appears, not how often
                word_proba_array[0][word_idx] += 1 / (spam_count_total)

        #calculating for spam=no rows
        for row_index in not_spam_rows_indices:
            word_count = word_count_matrix[row_index][word_idx]
            if(word_count > 0): #only checks if the word appears, not how often
                word_proba_array[1][word_idx] += 1 / (not_spam_count_total)
                #-> same as before but for spam=no mails

    probabilities_df = pd.DataFrame({
        'Word': words,
        'P(Word|Spam)': word_proba_array[0],
        'P(Word|NotSpam)': word_proba_array[1]
    })

    return probabilities_df


def get_cond_probas_attempt_2(p_mails:pd.DataFrame, p_labels:pd.DataFrame):

    cv = CountVectorizer()
    word_count_matrix = cv.fit_transform(p_mails.to_numpy()).toarray()

    alpha = 1
    unique_word_count = len(cv.get_feature_names())
    words = np.array(cv.get_feature_names())

    #instead of looping through the whole dataset counting word appearcances and adding probabilities, make 
    #an array that counts each word's appearances over the whole dataset (also considers how often a word
    # appears in a sentence and not only if or if it doean's appear as in the 1.attempt)
    spam_word_counts = word_count_matrix[p_labels == 1].sum(axis=0)
    not_spam_word_counts = word_count_matrix[p_labels == 0].sum(axis=0)

    #Calculating the contiditional probalbilities by going  spam_word_counts/ total amount of spam word
    # + it adds laplace smoothing as seen here:  https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece
    word_proba_spam = (spam_word_counts + alpha) / (spam_word_counts.sum() + alpha * unique_word_count)
    word_proba_not_spam = (not_spam_word_counts + alpha) / (not_spam_word_counts.sum() + alpha * unique_word_count)

    probabilities_df = pd.DataFrame({
        'Word': words,
        'P(Word|Spam)': word_proba_spam,
        'P(Word|NotSpam)': word_proba_not_spam
    })

    return probabilities_df


In [14]:
#GIve it a test

data_path = "Data/Preprocessing/data_cooked.csv"

dataframe = pd.read_csv(data_path)

# somehow there are missing values somewhere, may have to check Preprocessing.ipynb again
# very important to reset index! Because pandas would otherwise memorize the old indices which is bad for what is about to come
dataframe = dataframe.dropna(subset=['Message']).reset_index(drop=True)

mails = dataframe['Message']
labels = dataframe['Category']

cond_probas = get_cond_probas_attempt_1(p_mails=mails,p_labels=labels)

cond_probas.head()

,Word,P(Word|Spam),P(Word|NotSpam)
0,aa,0.000946,0.003380
1,aaa,0.000000,0.000654
2,aaaenerfax,0.000000,0.000109
3,aadedeji,0.000000,0.000109
4,aagraw,0.000000,0.000109


In [15]:
#Just out of curiosity some smaple conditional probabilities
examples = ['free', 'call', 'friend', "tomorrow", 'signup', 'money', 'win', 'verif', 'confirm']

selected_probabilities = cond_probas[cond_probas['Word'].isin(examples)]

print(selected_probabilities)



           Word  P(Word|Spam)  P(Word|NotSpam)
3417       call      0.217967         0.135194
4867    confirm      0.017967         0.034344
9150       free      0.195272         0.055495
9216     friend      0.034988         0.024313
15768     money      0.133806         0.015373
22408    signup      0.004728         0.000109
24994  tomorrow      0.009929         0.034235
26319     verif      0.008983         0.001090
27228       win      0.046809         0.005233


Now on slide number 45: Classifying spam:

In [16]:
#Method for classifying a single message

def classify_message(message, cond_probas, prior_probas):
    

    #the next line make a countvectorizer that uses the same 
    #columns names as in the training (column names = unique words)
    #this is done by adding the same vocabulary as in Word
    cv = CountVectorizer(vocabulary=cond_probas['Word'].values)

    #with this CountVectorizer you can make a vector from the message
    #same as the word_count_matrix, but only for one mail text
    message_vector = cv.transform([message]).toarray().flatten()

    #for the calculation it's somehow better to use log of the actual probabs
    #avoids some kind of problem that arieses if one calculates with very small numbers
    log_prob_spam = np.log(prior_probas['P_spam'])
    log_prob_not_spam = np.log(prior_probas['P_not_spam'])

    #in this loop the calculation for the vector is done for spam=yes as well as for spam=no
    for word, count in zip(cv.get_feature_names(), message_vector): 
        if count > 0:  
            if word in cond_probas['Word'].values: #could be that test data contains words that haavent been in training data
                word_data = cond_probas[cond_probas['Word'] == word] # get the word's conditional probabilities
                log_prob_spam += count * np.log(word_data['P(Word|Spam)'].values[0]) 
                log_prob_not_spam += count * np.log(word_data['P(Word|NotSpam)'].values[0])

    #make the choice by comparing the two values
    if log_prob_spam > log_prob_not_spam:
        return 1
    else:
        return 0

In [17]:
#Warning
# This cell may take a few minutes: 
# it loads the data, trains the model, makes predictions and calculates a score of the predictions

#using train_test_split to make a train and a test set
from sklearn.model_selection import train_test_split

data_path = "Data/Preprocessing/data_cooked.csv"
dataframe = pd.read_csv(data_path)
dataframe = dataframe.dropna(subset=['Message']).reset_index(drop=True)

mails = dataframe['Message']
labels = dataframe['Category']

#making a train set which is used to calculate the probabilities and a test set to test afterwards with non seen cases
mails_train, mails_test, labels_train, labels_test = train_test_split(
    mails, labels, test_size=0.25, random_state=42,stratify=labels) #random_state shuffles all lines in the dataset, stratify balances the calsses spam and no spam because there's a lot more spam=no than spam=yes in our data

#calling our functions from earlier
prior_probas = get_prior_probas(mails_train, labels_train)

#Decide if you use get_cond_probas_attempt_1 or get_cond_probas_attempt_2 HERE:
conditional_probas = get_cond_probas_attempt_2(mails_train, labels_train)

#creating the result array with a list comprehension for each test mail in mails_test
results = [classify_message(msg, conditional_probas, prior_probas) for msg in mails_test]


#calculate mean of an array that looks like this [True,False,False,True,True,True...]
#using a list comprehension that zips together the model's predictions in results and the actual
#labels in labels_test
#then with pred == real it creates a True/False value, indicating, if the prediction was right/wrong
#finally using np.mean to create the average
accuracy = np.mean([pred == real for pred, real in zip(results, labels_test)])
print(f"Accuracy is: {accuracy:.2%}")



## Another NBC example

In [ ]:
#importing the packages and data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB 

data_path = "Data/Preprocessing/data_cooked.csv"
dataframe = pd.read_csv(data_path)

# There were some null values in the Message column so Idropped these 
dataframe = dataframe.dropna(subset=['Message']) 

dataframe.groupby('Category').describe() 

In [ ]:
# create train/test split

x_train, x_test, y_train, y_test = train_test_split(dataframe.Message, dataframe.Category)


# get word count and store the data as a matrix 
cv = CountVectorizer()
x_train_count = cv.fit_transform(x_train.values)

x_train_count 

In [ ]:
# train model
naiveBayes = MultinomialNB()
naiveBayes.fit(x_train_count, y_train) 

In [ ]:
#pre-test for ham
email_ham = ["hey wanna meet up for dinner tonight?"]
email_ham_count = cv.transform(email_ham)
naiveBayes.predict(email_ham_count) 

In [ ]:
#pre-test for spam
email_spam = ["reward money insurance click win"]
email_spam_count = cv.transform(email_spam)
naiveBayes.predict(email_spam_count) 

In [ ]:
# test model 
x_test_count = cv.transform(x_test)
naiveBayes.score(x_test_count, y_test)

## Deep Learning Model with tensorflow ##

Important: to use tensorflow, you'll have to

1. conda remove --name spam_classifier_env --all (to remove the old environment)
2. conda env create -f environment.yml (to create the new enviroment containing keras)
3. conda activate spam_classifier_env (to activate the new environment)
4. restart notebook



In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


#importing the packages and data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 


In [ ]:
#Load data and prepare
data_path = "Data/Preprocessing/data_cooked.csv"
dataframe = pd.read_csv(data_path)
dataframe = dataframe.dropna(subset=['Message']).reset_index(drop=True)


X = dataframe['Message']
y = dataframe['Category']

cv = CountVectorizer()
X = cv.fit_transform(X).toarray()  # make it a matrix, the network can handle (not sparse matrix)


#this will tell how the input shape of the network needs to look like
print(len(cv.get_feature_names()))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, input_shape=[X_train.shape[1]], activation="relu"),
    tf.keras.layers.Dense(128,  activation="relu"),
    tf.keras.layers.Dense(128,  activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])


In [ ]:
#Takes a while, reduce epochs do less training

# Model training
history = model.fit(
    X_train, 
    y_train, 
    epochs=3, 
    batch_size=32, 
    validation_data=(X_test, y_test)
)

# Evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy on test data: {accuracy*100:.2f}%")

